# FLOYao.jl

A backend to efficiently simulate fermionic linear optics (FLO) circuits in [Yao.jl](https://github.com/QuantumBFS/Yao.jlhttps://github.com/QuantumBFS/Yao.jl) based on [Classical simulation of noninteracting-fermion quantum circuits](https://arxiv.org/abs/quant-ph/0108010).

## Contents
 - [Installation](#Installation)
 - [Basic usage](#Basic-usage)
 - [List of supported gates](#List-of-supported-gates)
 - [Example: Transverse field Ising model](#Example:-Transverse-Field-Ising-model)
 - [Adding support for your own gates](#Adding-support-for-your-own-gates)
 - [Background: Fermionic linear optics circuits](#Background:-Fermionic-linear-optics-circuits)
 - [Known restrictions](#Known-restrictions)

## Installation
To install `FLOYao` open the julia REPL and type the following:
```julia
pkg> add FLOYao
```

## Basic usage
The heart of `FLOYao` is the `MajoranaReg` register type, which efficiently represents a state that is a [FLO unitary](#Background:-Fermionic-linear-optics-circuits) applied to the vacuum state $|0⋯0⟩$

First import `Yao` and `FLOYao`:

In [24]:
using Yao, FLOYao

then build a (here somewhat arbitrary) circuit consisting only of [FLO gates](#Background:-Fermionic-linear-optics-circuits)

In [4]:
nq = 4
θ = π/8
circuit = chain(nq)

push!(circuit, put(nq, 3=>Rz(0.5)))

xxg1 = kron(nq, 1 => X, 2 => X)
rg = rot(xxg1, θ)
push!(circuit, rg)  

xxg2 = kron(nq, 2 => X, 3 => Z, 4 => X)
rg = rot(xxg2, θ)
push!(circuit, rg)  
push!(circuit, put(nq, 3=>Rz(0.5)))
push!(circuit, put(nq, 1=>Z))

xxg3 = kron(nq, 2 => X, 3 => X)
rg = rot(xxg3, θ)

circuit

nqubits: 4
chain
├─ put on (3)
│  └─ rot(Z, 0.5)
├─ rot(nqubits: 4
kron
├─ 1=>X
└─ 2=>X, 0.39269908169872414)
├─ rot(nqubits: 4
kron
├─ 2=>X
├─ 3=>Z
└─ 4=>X, 0.39269908169872414)
├─ put on (3)
│  └─ rot(Z, 0.5)
└─ put on (1)
   └─ Z


and define an observable that is a sum of squares of Majorana operators

In [5]:
hamiltonian = xxg1 + xxg2 + xxg3 + kron(nq, 2=>Z) + kron(nq, 3=>Z)

nqubits: 4
+
├─ kron
│  ├─ 1=>X
│  └─ 2=>X
├─ kron
│  ├─ 2=>X
│  ├─ 3=>Z
│  └─ 4=>X
├─ kron
│  ├─ 2=>X
│  └─ 3=>X
├─ kron
│  └─ 2=>Z
└─ kron
   └─ 3=>Z


and finally create a register in the computational zero state via

In [6]:
mreg = MajoranaReg(nq)

MajoranaReg{Float64} with 4 qubits:
8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

Applying the circuit to the register works then exactly the same way as for a normal Array register:

In [7]:
apply(mreg, circuit)

MajoranaReg{Float64} with 4 qubits:
8×8 Matrix{Float64}:
 -1.0  -0.0       -0.0       -0.0       -0.0       -0.0       -0.0       -0.0
 -0.0  -0.92388    0.382683  -0.0       -0.0       -0.0       -0.0       -0.0
  0.0   0.382683   0.92388    0.0        0.0        0.0        0.0        0.0
  0.0   0.0        0.0        0.92388    0.0        0.0        0.382683   0.0
  0.0   0.0        0.0        0.0        0.540302   0.841471   0.0        0.0
  0.0   0.0        0.0        0.0       -0.841471   0.540302   0.0        0.0
  0.0   0.0        0.0       -0.382683   0.0        0.0        0.92388    0.0
  0.0   0.0        0.0        0.0        0.0        0.0        0.0        1.0

and the same goes for expectation values of observables

In [8]:
expval = expect(hamiltonian, mreg => circuit)

1.8535533905932737

or even gradients of these expectation values with respect to the circuit parameters

In [9]:
inδ, paramsδ = expect'(hamiltonian, mreg => circuit)

MajoranaReg{Float64}(4) => [0.0, -0.3535533905932738, -0.3535533905932738, 0.0]

and just to check that this is all consistent with running a full wavefunction simulation, can apply the same circuit on an `ArrayReg` and compare the expectation values and gradients

In [10]:
areg = zero_state(nq)
expval_full = expect(hamiltonian, areg => circuit)
expval_full ≈ expval

true

In [11]:
inδ_full, paramsδ_full = expect'(hamiltonian, areg => circuit)
paramsδ ≈ paramsδ_full

true

## List of supported gates

The following gates are FLO gates and supported by `FLOYao.jl`:

|  Gate | Comment |
|-------|---------|
|  `XGate`   | Together with `Y` the only gate that does not preserve fermionic parity |
|  `YGate`   |   See above  |
|  `ZGate`   |         |
|  `RotationGate{⋯,⋯,YGate}`  | The only single qubit rotation gate since $R_x(θ)γ_i R_x(-θ)$ is not a linear combination of Majorana operators for all Majorana operators. Similar for $R_y$ |
| `PauliKronBlock` | A kronecker product of Pauli operators s.t. that first and last operator are either $X$ or $Y$ and all operators in between are $Z$.  |
| `RotationGate{⋯,⋯,PauliKronBlock}` | A rotation gate with generator as in the last line. |
| `AbstractMatrix` | Unless the gate type is already explicitely implemented or know to not be a FLO gate, `FLOYao` will try to automatically convert the gate matrix in the qubit basis to a matrix in the Majorana basis. But note that this is fairly slow (although still poly-time instead of exp-time) |


## Example: VQE for the Transverse Field Ising model

For a more realistic use case, we have a look at VQE for the Transverse Field Ising model on a line whose Hamiltonian is given as 
$$
    H = J ∑_i^{L-1} X_i X_{i+1} + h ∑_i^L Z_i = U + T.
$$
and as Ansatz circuits we use the Hamiltonian Variational Ansatz
$$
    U(\vec θ) = ∏_i^p e^{-iθ_{i,U} U} e^{-iθ_{i,T} T} 
$$
with the initial state being the groundstate of the TFIM at $J = 0$, $|ψ_i⟩ = |0⋯0⟩$

In [13]:
# note that this is far beyond what is possible with a full wavefunction simulation
L = 100 
J = 0.5
h = 1.
p = 10

U = map(1:L-1) do i
    J * kron(L, i => X, i+1 => X)
end |> sum

T = map(1:L) do i
    h * kron(L, i => Z)
end |> sum

hamiltonian = T + U

circuit = chain(L)
for _ in 1:p
    for i in 1:L-1
        push!(circuit, rot(kron(L, i => X, i+1 => X), 0.))
    end
    for i in 1:L
        push!(circuit, put(L, i => Rz(0.)))
    end
end
nparams = nparameters(circuit)
dispatch!(circuit, rand(nparams) ./ 100)

ψ_i = MajoranaReg(L);

now that we defined the hamiltonian, the ansatz circuit and the initial state we can perform
simple gradient descent on the energy expectation value to find an approximation to the
groundstate of $H$:

In [15]:
iterations = 100
γ = 2e-2

for i in 1:iterations
    _, grad = expect'(hamiltonian, ψ_i => circuit)
    dispatch!(-, circuit, γ * grad)
    println("Iteration $i, energy = $(expect(hamiltonian, ψ_i => circuit))")
end

Iteration 1, energy = -85.58442028406317
Iteration 2, energy = -85.59679211578971
Iteration 3, energy = -85.60890593432809
Iteration 4, energy = -85.6207678543884
Iteration 5, energy = -85.63238391408974
Iteration 6, energy = -85.64376007967587
Iteration 7, energy = -85.65490224875602
Iteration 8, energy = -85.66581625226901
Iteration 9, energy = -85.67650785536361
Iteration 10, energy = -85.68698275738363
Iteration 11, energy = -85.69724659113479
Iteration 12, energy = -85.70730492160064
Iteration 13, energy = -85.71716324426018
Iteration 14, energy = -85.72682698314559
Iteration 15, energy = -85.73630148876207
Iteration 16, energy = -85.74559203597492
Iteration 17, energy = -85.75470382195283
Iteration 18, energy = -85.76364196423859
Iteration 19, energy = -85.77241149900364
Iteration 20, energy = -85.78101737952564
Iteration 21, energy = -85.78946447491538
Iteration 22, energy = -85.7977575691069
Iteration 23, energy = -85.80590136011091
Iteration 24, energy = -85.81390045952519
Ite

## Adding support for your own gates

Natively, the only FLO gates that come already shipped with `Yao.jl` are gates of
the form `kron(nq, i => σ1, i+1 => Z, ⋯, i-1 => Z, j => σ2)` and 
`rot(kron(nq, i => σ1, i+1 => Z, ⋯, i-1 => Z, j => σ2), θ)` where `σ1, σ2 ∈ [X,Y]`. But there are many more FLO gates, one being for example the $FSWAP$ gates which swaps to qubits while making sure to preserve the fermionic commutation relations

In [16]:
@const_gate FSWAP::ComplexF64 = [1 0 0 0; 0 0 1 0; 0 1 0 0; 0 0 0 -1]

If a gate defines a matrix representation, as we just did for the `FSWAP`gate, `FLOYao` supports them out of the box by manually checking if they are a FLO gate and then computing its matrix representation in the Majorana basis. But this method is fairly slow, compared to directly implementing `unsafe_apply!(::MajoranaReg, ::YourBlock)` and  `instruct!(::MajoranaReg, ::YourBlock)` and will warn you accordingly

In [18]:
nq = 4
fswap = put(nq, (1, 2) => FSWAP)
mreg = MajoranaReg(nq)
mreg |> put(nq, 2 => X)
mreg |> fswap

┌ Warning: Calling manual instruct!(MajoranaReg{Float64}(4), ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im -1.0 + 0.0im], (1, 2)).
│ You can greatly speed up your FLO gates by exactly implementing unsafe_apply!()
│ and instruct!() for them. See FLOYao/src/instruct.jl for how to do that.
└ @ FLOYao /home/janlukas/PhD/Work/code/FLOYao/src/instruct.jl:62


MajoranaReg{Float64} with 4 qubits:
8×8 Matrix{Float64}:
 -2.40901e-16  -2.94663e-16  -1.0          …   0.0   0.0   0.0   0.0
  3.57633e-16  -4.32975e-16  -1.0529e-16       0.0   0.0   0.0   0.0
 -1.0           1.20422e-17   2.61492e-16      0.0   0.0   0.0   0.0
 -7.05281e-17  -1.0           2.65282e-16      0.0   0.0   0.0   0.0
 -0.0          -0.0          -0.0             -1.0  -0.0  -0.0  -0.0
 -0.0          -0.0          -0.0          …  -0.0  -1.0  -0.0  -0.0
 -0.0          -0.0          -0.0             -0.0  -0.0  -1.0  -0.0
 -0.0          -0.0          -0.0             -0.0  -0.0  -0.0  -1.0

now before we fix these warnings, let's see how long the current implementation takes:

In [19]:
using BenchmarkTools
using Suppressor # we don't want to get all the warnings when benchmarking
@benchmark @suppress apply!($mreg, $fswap)

BenchmarkTools.Trial: 2319 samples with 1 evaluation.
 Range (min … max):  929.353 μs … 24.616 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.738 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):     2.142 ms ±  1.651 ms  ┊ GC (mean ± σ):  1.59% ± 5.83%

       █▅▂▁▁                                                    
  ▇▅▁▅▇██████▆▇▆▇▇█▆▇▆▆▅▆▆▇▆▆▅▃▅▅▅▄▃▅▃▁▅▅▆▆▄▁▃▃▁▄▃▃▃▃▃▁▁▃▃▃▁▁▄ █
  929 μs        Histogram: log(frequency) by time      9.55 ms <

 Memory estimate: 339.19 KiB, allocs estimate: 496.

To find out what the matrix representation of the `FSWAP` gate in the Majorana basis is, it is easiest to retrace what is happening inside `instruct!(::MajoranaReg, ::AbstractMatrix, locs)`

In [20]:
W = FLOYao.qubit2majoranaevolution(Matrix(fswap.content), fswap.locs)

4×4 Matrix{Float64}:
 -2.40901e-16  -2.94663e-16  -1.0           2.35127e-16
  3.57633e-16  -4.32975e-16  -1.0529e-16   -1.0
 -1.0           1.20422e-17   2.61492e-16  -2.81188e-16
 -7.05281e-17  -1.0           2.65282e-16   3.00191e-16

In [21]:
matlocs = 2*(fswap.locs[1]-1)+1:2(fswap.locs[end])

1:4

this matrix gets left-multiplied to the columns `1:4` in the last line of `FLOYao.majorana_unsafe_apply!(::MajoranaReg, ::PutBlock)`. So we can instead implement the action of an `FSWAP` gate on a `MajoranaReg` directly as follows:

In [22]:
function YaoBlocks.unsafe_apply!(reg::MajoranaReg, b::PutBlock{2,2,FSWAPGate})
    FLOYao.areconsecutive(b.locs) || throw(NonFLOException("FSWAP must act on consecutive qubits"))
    instruct!(reg, Val(:FSWAP), b.locs)
end

function Yao.instruct!(reg::MajoranaReg, ::Val{:FSWAP}, locs::Tuple)
    i1, i2 = locs
    ψ1, ψ2 = reg.state[2i1-1,:], reg.state[2i1,:]
    ψ3, ψ4 = reg.state[2i2-1,:], reg.state[2i2,:]
    reg.state[2i1-1,:] .=  .-ψ3
    reg.state[2i1,:] .=  .-ψ4
    reg.state[2i2-1,:] .=  .-ψ1
    reg.state[2i2,:] .=  .-ψ2
    return reg
end

In [23]:
@benchmark apply!($mreg, $fswap)

BenchmarkTools.Trial: 10000 samples with 560 evaluations.
 Range (min … max):  206.909 ns …   4.917 μs  ┊ GC (min … max): 0.00% … 90.06%
 Time  (median):     222.789 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   258.945 ns ± 261.773 ns  ┊ GC (mean ± σ):  7.68% ±  7.30%

  ▇▇▅█▂▃▃▂                                             ▁▁       ▁
  █████████▇▆▇▇▆▆▅▆▅▅▄▄▃▄▄▅▃▅▄▅▅▂▄▅▅▅▄▅▅▄▄▆▆▇▇▇▇▇▇▇▆▆▆▇████▇▇▇▆ █
  207 ns        Histogram: log(frequency) by time        515 ns <

 Memory estimate: 512 bytes, allocs estimate: 4.

Now, that is quite a significant speedup!

## Background: Fermionic linear optics circuits

This section is more here, to fix the convention of
[Jordan-Wigner transform](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation)
and [Majorana operators](https://en.wikipedia.org/wiki/Majorana_fermion) that we use here,
and less to explain the full theory behind those. For the latter, we, once again, recommend [Classical simulation of noninteracting-fermion quantum circuits](https://arxiv.org/abs/quant-ph/0108010).

We define the Majorana operators $γ_i$ via 
$$
    γ_{2i-1} = ∏_{j=1}^{i-1} (-Z_j) X_i
$$
and
$$
    γ_{2i} = ∏_{j=1}^{i-1} (-Z_j) Y_i.
$$
This implies the normal fermionic creation and annihilation operators are given by
$$
    c_j = \frac{1}{2} (γ_{2j-1} + iγ_{2j})
    \quad \textrm{and} \quad
    c_j^† = \frac{1}{2} (γ_{2j-1} - iγ_{2j})
$$
and products of two Majorana operators are of the form
$$
    σ_i \left(∏_{i<j<k} Z_k \right) σ_k
    \quad \textrm{or} \quad
    Z_i
$$
with $σ_i, σ_k ∈ \{X, Y\}$.

Any unitary that takes all Majorana operators to a linear combination of Majorana operators
under conjugation, i.e. that satisfies
$$
    U γ_i U^† = R_i^j γ_j
$$
with some $R ∈ O(2n)$ is a FLO unitary. In particular, if a unitary is of the form 
$$
    U = e^{-iθH}
$$
with 
$$
    H = \frac{i}{4} \sum_{i,j} H^{ij} γ_i γ_j
$$
it is a FLO unitary with even $R ∈ SO(2n)$.

But note, that not all FLO unitaries are of that form. For example $X_i$ is also a FLO 
gate since it either commutes or anti-commutes with all Majorana operators, but the associated
matrix $R$ always has determinant $-1$.

Calculating the expectation values of hamiltonians like the one above when evolving the 
vacuum state with FLO circuits is efficiently possible. Simply evolve the 
Hamiltonian in the Heisenber picture to
$$
    UHU^† = \frac{i}{4} R^{m}_{i} R^{n}_{j} H^{ij} γ_{m} γ_{n} 
           =: \frac{i}{4} \tilde H^{mn} γ_{m} γ_{n}.
$$

For the expectation value this makes then
$$
\begin{aligned}
    ⟨ψ|UHU^†|ψ⟩ &= \frac{i}{4} \tilde H^{mn} ⟨Ω|γ_{m} γ_{n}|Ω⟩ \\
                &= - \frac{1}{2} ∑_{i} \tilde H^{2i-1,2i} \\
                &= - \frac{1}{2} ∑_{i>k} R^{2i-1}_{m} R^{2i}_{n} H^{mn} \\
\end{aligned}
$$
where we used in from the first to second line one needs to carefully think which of the 
$⟨Ω|γ_{m} γ_{n}|Ω⟩$ are zero and which cancel each other out due to the anti-symmetry of $H^{mn}$.

## Known restrictions

###  Expectation values of higher order observables
So far, `FLOYao` only supports expectation values of observables that are sums of squares of 
Majorana operators. But in general, one can use [Wick's theorem](https://en.wikipedia.org/wiki/Wick%27s_theorem) to calculate the expectation values of expressions of the form 
$$
    ⟨ψ_i|O|ψ_i⟩
$$
where $|ψ_i⟩$ is a computational basis state and $O$ a string of Majorana operators and thus using linearity of the expectation value it is possible to efficiently calculate the expectation value of any observable that can be expanded in a sum of polynomially (in the number of qubits) many products of Majorana operators. (See also [Classical simulation of noninteracting-fermion quantum circuits](https://arxiv.org/abs/quant-ph/0108010) again for details). If you need expectation values of higher order (in the number of Majorana operators involved) observables, feel free to open a PR!

### "Hidden" FLO circuits
`Yao.jl` allows to express the same circuit in different forms. E.g. 
```julia
    chain(nq, put(1 => X), put(2 => X))
```
and
```julia
    kron(nq, 1 => X, 2 => X)
```
represent the same FLO circuit, but only the latter will be recognised as such. Similarly 
```julia
     kron(nq, 1 => X, 2 => X, 3 => X, 4 => X)
```
and
```julia
     chain(nq, kron(1 => X, 2 => X), kron(3 => X, 4 => X))
```
represent the same FLO circuit but only the latter will be recognised as such. This is because

 - We don't check if a whole `ChainBlock` is a FLO circuit, even if its single
   gates are not. Instead a `ChainBlock` is applied gate by gate, each of which
   has to be a FLO gate.
 - For `KronBlock`s we first try if each factor is a FLO gate and then if the whole
   block altogether is of the form  `kron(nq, i => σ1, i+1 => Z, ⋯, i-1 => Z, j => σ2)`
   with `σ1, σ2 ∈ [X, Y]`.

If you run into a case that is a FLO circuit / gate but not recognised as such please open an issue or even pull request.
